In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Funkcje analityczne umożliwiają wykonywanie zapytań poza bieżącym wierszem. Są przydatne zwłaszcza wtedy, gdy trzeba porównać bieżący wiersz z innymi wierszami w tabeli.

Funkcje analityczne mogą służyć do identyfikowania strony wyjściowej sesji (przy użyciu FIRST_VALUE z podziałem trafień sesji, uporządkowanych malejąco według zdarzenia_sygnatury czasowej ) oraz ścieżek nawigacji do i ze stron (przy użyciu LAG i LEAD z kluczem page_location

Zrozumienie operatora PARTITION BY jest kluczem do zrozumienia funkcji analitycznych. PARTITION BY tworzy partycję podzbioru danych pogrupowanych i uporządkowanych według kolumn podanych jako dane wejściowe. Następnie pobierasz wartości z tej partycji za pomocą funkcji analitycznych.

In [ ]:
%%bigquery --project 

select
    user_pseudo_id,
    key,
    value.string_value
from
    `ga.events_20210601`
    cross join unnest(user_properties)
where 
    user_pseudo_id = '1467032112.1598377364'
group by 
    user_pseudo_id,
    key,
    value.string_value

,user_pseudo_id,key,string_value
0,1467032112.1598377364,custom_client_id,1467032112.1598377364.
1,1467032112.1598377364,visitor_status,buyer


CROSS JOIN można zastąpić przecinkiem

https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax#cross_join

In [ ]:
%%bigquery --project 

select
    user_pseudo_id,
    key,
    value.string_value
from
    `ga.events_20210601`,
    unnest(user_properties) as user_properties
where 
    user_pseudo_id = '1467032112.1598377364'
group by 
    user_pseudo_id,
    key,
    value.string_value

,user_pseudo_id,key,string_value
0,1467032112.1598377364,custom_client_id,1467032112.1598377364.
1,1467032112.1598377364,visitor_status,buyer


In [ ]:
%%bigquery --project 

select
    user_pseudo_id,
    value.string_value as visitor_status
from
     `ga.events_20210601`,
    unnest(user_properties)
where 
    key = 'visitor_status'
group by 
    user_pseudo_id,
    value.string_value
LIMIT 3

,user_pseudo_id,visitor_status
0,745382386.1622551885,buyer
1,wMtxAgmNI7oJiVbjsHq8lW6+z2/bA4gnjGexA99/a6o=.1...,buyer
2,992224805.1622466892,buyer


Podzapytanie skalarne

https://cloud.google.com/bigquery/docs/reference/standard-sql/subqueries#scalar_subquery_concepts

In [ ]:
%%bigquery --project 

# To jest podzapytanie skalarne

select
    user_pseudo_id,
    max((select value.string_value from unnest(user_properties) where key = 'visitor_status')) as visitor_status
from
    `ga.events_20210601`
group by 
    user_pseudo_id
having 
    visitor_status is not null
LIMIT 3

,user_pseudo_id,visitor_status
0,745382386.1622551885,buyer
1,wMtxAgmNI7oJiVbjsHq8lW6+z2/bA4gnjGexA99/a6o=.1...,buyer
2,992224805.1622466892,buyer


Count Page Views By Pages And Titles
Query GA4 Data In Go

In [ ]:
%%bigquery --project

select
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_title') as page_title,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    count(event_name) as pageviews
from
    `ga.events_20210601`
where
    event_name = 'page_view'
group by 
    page_title,
    page
order by 
    pageviews desc

Count Unique Page Views

In [ ]:
%%bigquery --project

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_title') as page_title,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page
from
    `ga.events_20210601`
where
    event_name = 'page_view')

select 
    page_title,
    page,
    count(*) as total_pageviews,
    count(distinct concat(user_pseudo_id,session_id)) as unique_pageviews
from 
    prep
group by 
    page_title,
    page
order by
    unique_pageviews desc

Identify Most Visited Landing Pages

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    case when (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'entrances') = 1 then true else false end as landing_page
from
    `ga.events_20210601`
where
    event_name = 'page_view')

select 
    case when landing_page is true then page else null end as landing_page,
    count(distinct concat(user_pseudo_id,session_id)) as entrances
from 
    prep
group by 
    landing_page
having 
    landing_page is not null
order by
    entrances desc

MAX & MIN 

MAX zwraca największą („maksymalną”) wartość danych wejściowych inną niż null. 

https://cloud.google.com/bigquery/docs/reference/standard-sql/aggregate_functions#max


COUNTIF

https://cloud.google.com/bigquery/docs/reference/standard-sql/aggregate_functions#countif

Jeśli warunek ma wartość true, licznik jest zwiększany.

In [ ]:
%%bigquery --project

select
    user_pseudo_id,
    timestamp_micros(user_first_touch_timestamp) as user_first_touch_timestamp,
    traffic_source.medium,
    max((select value.string_value from unnest(user_properties) where key = 'visitor_status')) as visitor_status,
    max((select value.string_value from unnest(user_properties) where key = 'custom_client_id')) as custom_client_id,
    max((select value.string_value from unnest(user_properties) where key = 'loyalty_level')) as loyalty_level,
    countif(event_name = 'session_start') as number_of_sessions
from
    `ga.events_20210601`
group by 
    user_pseudo_id,
    user_first_touch_timestamp,
    medium
order by
    number_of_sessions desc
LIMIT 3

,user_pseudo_id,user_first_touch_timestamp,medium,visitor_status,custom_client_id,loyalty_level,number_of_sessions
0,913454443.1586958451,NaT,referral,buyer,913454443.1586958451.,gold,18
1,1506330390.1613083013,2021-02-11 22:37:50.374756+00:00,(none),buyer,1506330390.1613083013.,gold,9
2,XrteJmQJ9coTtILcUszFksLez39ZPXJQi4aj3FoyBkQ=.1...,2021-05-31 07:32:03.226805+00:00,(none),buyer,580444156.1622533679.,gold,7


IFNULL

https://cloud.google.com/bigquery/docs/reference/standard-sql/conditional_expressions#ifnull

Tam gdzie jest NULL daj wartość zastępczą

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
    max((select value.string_value from unnest(event_params) where key = 'medium')) as medium,
    max((select value.string_value from unnest(event_params) where key = 'source')) as source,
from
    `ga.events_20210601`
group by
    user_pseudo_id,
    session_id)

select
# Tam gdzie jest NULL wstaw (direct)
    concat(ifnull(source,'(direct)'),' / ',ifnull(medium,'(none)')) as session_source_medium,
    count(distinct concat(user_pseudo_id,session_id)) as sessions,
    count(distinct case when session_engaged = '1' then concat(user_pseudo_id,session_id) end) as engaged_sessions,
    count(distinct user_pseudo_id) as users,
    count(distinct case when engagement_time_msec > 0 then user_pseudo_id end) as active_users
from
    prep
group by 
    session_source_medium
order by 
    sessions desc

Identify The Exit Page Of A Single Session

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    case when (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'entrances') = 1 then true else false end as landing_page
from
    `ga.events_20210601`
where
    event_name = 'page_view')

select 
    case when landing_page is true then page else null end as landing_page,
    count(distinct concat(user_pseudo_id,session_id)) as entrances
from 
    prep
group by 
    landing_page
having 
    landing_page is not null
order by
    entrances desc

Identify The Exit Page Of A Single Session

Funkcje analityczne

https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions

In [ ]:
%%bigquery --project 

SELECT
    distinct device.category as device,
    platform,
    count(distinct concat(user_pseudo_id, (select value.int_value from unnest(event_params) where key = 'ga_session_id'))) as sessions,
    max(count(distinct concat(user_pseudo_id, (select value.int_value from unnest(event_params) where key = 'ga_session_id')))) over () as max_sessions_over_null,
    max(count(distinct concat(user_pseudo_id, (select value.int_value from unnest(event_params) where key = 'ga_session_id')))) over (partition by platform) as max_sessions_over_platform,
    max(count(distinct concat(user_pseudo_id, (select value.int_value from unnest(event_params) where key = 'ga_session_id')))) over (partition by device.category) as max_sessions_over_device,
    max(count(distinct concat(user_pseudo_id, (select value.int_value from unnest(event_params) where key = 'ga_session_id')))) over (partition by platform, device.category) as max_sessions
FROM
 `ga.events_20210601`
group by
    device,
    platform
order by
    sessions desc

,device,platform,sessions,max_sessions_over_null,max_sessions_over_platform,max_sessions_over_device,max_sessions
0,desktop,WEB,6885,6885,6885,6885,6885
1,mobile,WEB,204,6885,6885,204,204
2,tablet,WEB,14,6885,6885,14,14


In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    event_timestamp
from
    `ga.events_20210601`
where
    event_name = 'page_view'
    and user_pseudo_id = '693452447.1622524545'
order by 
    event_timestamp)

select 
    user_pseudo_id,
    session_id,
    page,
    event_timestamp,
    first_value(concat(page,event_timestamp)) over (partition by user_pseudo_id,session_id order by event_timestamp desc) as exit_page
from 
    prep 
order by 
    event_timestamp

Identify Most Popular Exit Pages

In [ ]:
%%bigquery --project

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    event_timestamp
from
    `ga.events_20210601`
where
    event_name = 'page_view'
order by 
    event_timestamp),

prep_exit as (
select 
    user_pseudo_id,
    session_id,
    page,
    event_timestamp,
    first_value(concat(page,event_timestamp)) over (partition by user_pseudo_id,session_id order by event_timestamp desc) as exit_page
from 
    prep 
order by 
    event_timestamp)

select
    case when concat(page,event_timestamp) = exit_page then page else null end as exit_page,
    count(distinct concat(user_pseudo_id,session_id)) as exits
from 
    prep_exit
group by 
    exit_page
having 
    exit_page is not null 
order by 
    exits desc

Identify Page Pathing For A Single Session

LAG pobiera wartość poprzedniego wiersza w partycji, a LEAD pobiera wartość następnego wiersza w partycji.

https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions#lead

In [ ]:
%%bigquery --project

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    event_timestamp
from
    `ga.events_20210601`
where
    event_name = 'page_view'
    and user_pseudo_id = '693452447.1622524545')

select
    user_pseudo_id,
    session_id,
    lag(page,1) over (partition by user_pseudo_id,session_id order by event_timestamp asc) as previous_page,
    page,
    lead(page,1) over (partition by user_pseudo_id,session_id order by event_timestamp asc) as next_page,
    event_timestamp
from 
    prep

Identify Most Popular Pathing For Any Page

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where event_name = 'page_view' and key = 'ga_session_id') as session_id,
    (select value.string_value from unnest(event_params) where event_name = 'page_view' and key = 'page_location') as page,
    event_timestamp
from
    `ga.events_20210601`
where
    event_name = 'page_view'),

prep_navigation as (
select
    user_pseudo_id,
    session_id,
    lag(page,1) over (partition by user_pseudo_id,session_id order by event_timestamp asc) as previous_page,
    page,
    lead(page,1) over (partition by user_pseudo_id,session_id order by event_timestamp asc) as next_page,
    event_timestamp
from 
    prep)

select 
    ifnull(previous_page,'(entrance)') as previous_page,
    page,
    ifnull(next_page,'(exit)') as next_page,
    count(distinct concat(user_pseudo_id, session_id)) as count
from 
    prep_navigation
where 
    page = 'https://www.example/'
group by 
    previous_page,
    page,
    next_page
having 
    page not in (previous_page,next_page)
order by 
    count desc
LIMIT 1